# Introduction

This notebook illustrates the relationship between power consumption and CPU load <br />
of a laptop computer. Sample power consumption and CPU load data is collected from <br />
a Macbook Pro having a Core i5 2.9 GHZ processor powered by MacOS Seirra. <br />

----
**Tools used:**

*MacOS SystemProfiler*: to collect battery discharge info., that is, 
                          voltage and current readings.
                          
*[Intel Power Gadget](https://software.intel.com/en-us/articles/intel-power-gadget-20)*: CPU frequency and power consumption monitoring tool

**Python Libraries**

*psutil*: to gather CPU load at user-defined sampling intervals

*pandas and numpy*: to process the resulting data

*bokeh*: for plotting

In [1]:
from power_profile import cpuLoadPower

In [2]:
cpuLoadPower(1000, 1) 


2017-07-11 12:38:09.140322 > 0 samples processed
2017-07-11 12:40:51.109189 > 100 samples processed
2017-07-11 12:43:35.216932 > 200 samples processed
2017-07-11 12:46:17.819553 > 300 samples processed
2017-07-11 12:49:04.339438 > 400 samples processed
2017-07-11 12:52:10.507287 > 500 samples processed
2017-07-11 12:54:55.536163 > 600 samples processed
2017-07-11 12:57:37.747262 > 700 samples processed
2017-07-11 13:00:28.483069 > 800 samples processed
2017-07-11 13:03:11.515793 > 900 samples processed


In [2]:
import pandas as pd

In [3]:
sys_data = pd.read_csv('volt-amp-chrg-cpuLoad.csv')
sys_data['Power'] = sys_data.mV * sys_data.mA * -1e-6   # Power = V * A

In [4]:
sys_data.head(5)

,Time,mV,mA,mAh,cpu_load,Power
0,12:38:09:139982,11819,-787,3669,5.7,9.301553
1,12:38:10:759085,11819,-787,3669,23.5,9.301553
2,12:38:12:390611,11819,-787,3669,21.4,9.301553
3,12:38:13:993952,11819,-787,3669,14.5,9.301553
4,12:38:15:628270,11819,-787,3669,13.6,9.301553


In [5]:
cols = 'System Time,CPU Frequency_0(MHz),Processor Power_0(Watt)'
cpu_data = pd.read_csv("PwrData_2017-7-11_12-38-14.csv", usecols=cols.split(','))
cpu_data.rename(columns={'System Time':'Time', 
                         'CPU Frequency_0(MHz)':'CPU_Freq_(MHz)', 
                         'Processor Power_0(Watt)': 'Processor_Power'}, inplace=True)
cpu_data.head()


,Time,CPU_Freq_(MHz),Processor_Power
0,12:38:14:240,1300.0,1.772
1,12:38:14:338,1300.0,1.747
2,12:38:14:442,1300.0,2.284
3,12:38:14:537,1300.0,2.038
4,12:38:14:637,1700.0,2.741


In [6]:
cpu_data.dropna(inplace=True)

In [7]:
import datetime

In [8]:
sys_data['Time'] = sys_data.Time.apply(lambda t: datetime.datetime.strptime('2017-07-11 ' + t[:-7], '%Y-%m-%d %H:%M:%S'))
cpu_data['Time'] = cpu_data.Time.apply(lambda t: datetime.datetime.strptime('2017-07-11 ' + t[:-4], '%Y-%m-%d %H:%M:%S'))

In [12]:
sys_data['Time'] = sys_data.Time.apply(lambda t: t.timestamp())
cpu_data['Time'] = cpu_data.Time.apply(lambda t: t.timestamp())

In [13]:
start_time = cpu_data.Time[0] if cpu_data.Time[0] < sys_data.Time[0] else sys_data.Time[0]

In [14]:
sys_data['Time'] = sys_data.Time - start_time
cpu_data['Time'] = cpu_data.Time - start_time

In [31]:
cpu_data.head()

,Time,CPU_Freq_(MHz),Processor_Power
0,5.0,1300.0,1.772
1,5.0,1300.0,1.747
2,5.0,1300.0,2.284
3,5.0,1300.0,2.038
4,5.0,1700.0,2.741


In [55]:
avg_cpu_data = cpu_data.groupby(by='Time', as_index=False).agg(np.mean)

In [56]:
avg_cpu_data.head()

,Time,CPU_Freq_(MHz),Processor_Power
0,5.0,1450.0,2.3365
1,6.0,2020.0,5.7563
2,7.0,1800.0,4.5900
3,8.0,1900.0,4.1122
4,9.0,2340.0,6.1323


In [60]:
merged_data = pd.merge(sys_data, avg_cpu_data, on='Time')

In [71]:
merged_data.head()

,Time,mV,mA,mAh,cpu_load,Power,CPU_Freq_(MHz),Processor_Power
0,6.0,11819,-787,3669,13.6,9.301553,2020.0,5.7563
1,8.0,11819,-787,3669,17.4,9.301553,1900.0,4.1122
2,9.0,11819,-787,3669,17.5,9.301553,2340.0,6.1323
3,11.0,11819,-787,3669,12.4,9.301553,1730.0,5.5018
4,13.0,11819,-787,3669,15.7,9.301553,1300.0,2.8642


In [16]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure



In [17]:
output_notebook()


Loading BokehJS ...

In [18]:
from bokeh.models import LinearAxis, Range1d


In [20]:
p = figure(plot_width=700, plot_height=400, toolbar_location="above")

p.line(sys_data['Time'], sys_data['cpu_load'], color="firebrick", legend='CPU load')

p.extra_y_ranges = {"pwr": Range1d(start=10, end=35)}
p.line(sys_data['Time'], sys_data['Power'], color="navy", y_range_name="pwr", legend='Power Consumption')

p.add_layout(LinearAxis(y_range_name="pwr"), 'right')


p.xaxis.axis_label = "Time (sec)"

p.yaxis[0].axis_label = "CPU load (%)"
p.yaxis[0].axis_label_text_color = "firebrick"
p.yaxis[0].major_label_text_color = "firebrick"
p.yaxis[0].major_tick_line_color = 'firebrick'
p.yaxis[0].minor_tick_line_color = 'firebrick'
p.yaxis[0].axis_line_color = 'firebrick'

p.yaxis[1].axis_label = "Power (Watt)"
p.yaxis[1].axis_label_text_color = "navy"
p.yaxis[1].major_label_text_color = "navy"
p.yaxis[1].major_tick_line_color = 'navy'
p.yaxis[1].minor_tick_line_color = 'navy'
p.yaxis[1].axis_line_color = 'navy'

p.legend.location = "top_center"

p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.6
p.background_fill_color = "whitesmoke"
show(p)


The above plot shows that whenever CPU load hikes so does power consumption and vise versa.
Interestingly, there seems to be a consistent time gap between the change in CPU load and 
power consumption. 


In [23]:
import numpy as np

In [74]:
# ===============================================
# Fitting from System Profiler data
fit_from_sys_data = np.polyfit(sys_data['cpu_load'], sys_data['Power'], 1)
fn_from_sys_data = np.poly1d(fit_from_sys_data)

p = figure(plot_width=600, plot_height=400, toolbar_location="above")

p.circle(
    sys_data.cpu_load, 
    sys_data.Power, 
    color="firebrick",
    line_color="firebrick",
    fill_alpha = 0.4,
    size = 8
)

p.line(
    sys_data.cpu_load, 
    fn_from_sys_data(sys_data.cpu_load), 
    color="firebrick", 
    legend="Power = 0.22 * CPU_Load + 12.57"
)


# ===============================================
# Fitting merged data: System Profiler +  Intel Power Gadget
fit_from_merged_data = np.polyfit(merged_data['cpu_load'], merged_data['Processor_Power'], 1)
fn_from_merged_data = np.poly1d(fit_from_merged_data)

p.circle(
    merged_data.cpu_load, 
    merged_data.Processor_Power, 
    color="navy",
    line_color="navy",
    fill_alpha = 0.4,
    size = 8
)
p.line(
    merged_data.cpu_load, 
    fn_from_merged_data(merged_data.cpu_load), 
    color="navy", 
    legend="Power = 0.26 * CPU_Load + 2.87"
)

p.grid.grid_line_dash = [2,2]
p.grid.grid_line_alpha = 0.6

p.xaxis.axis_label = "CPU LOAD (%)"
p.yaxis.axis_label = "POWER (Watt)"
p.legend.location = "top_left"
p.background_fill_color = "whitesmoke"
show(p)

The linear line in the above plot shows that the power consumption is about $6$ Watts <br /> 
even when the computer is idle. Of course, the CPU is not the only component that consumes power.<br />
Power consumption ranges between $6$ and $35$ Watts as the CPU load goes from $0$ to $100\%$.